In [2]:
%load_ext autoreload
%autoreload 2

In [23]:
import torch
from torch.nn import Embedding
import tagme
from canlpy.helpers.ernie_helpers import load_name_to_QID,load_QID_to_eid, process_sentences

from canlpy.core.components.tokenization import BertTokenizer
from canlpy.core.models.ernie.model import ErnieForMaskedLM
from canlpy.previous_models.knowledge_bert.modeling import BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
#import logging
#logging.basicConfig(level=logging.INFO)

device = 'cpu'

KNOWLEDGE_DIR = '../canlpy/knowledge/ernie/'
PRE_TRAINED_DIR = '../canlpy/pretrained_models/ernie/ernie_base/'

NAME_TO_QID_FILE = KNOWLEDGE_DIR+ 'entity_map.txt'
QID_TO_EID_FILE = KNOWLEDGE_DIR+ 'entity2id.txt'
EID_TO_VEC_FILE = PRE_TRAINED_DIR + 'entity2vec.pt'

# Set the authorization token for subsequent calls.
tagme.GCUBE_TOKEN = "3aaa1f03-b732-49a1-9328-7f86cdaa13df-843339462"

In [26]:
model_original,_ = BertForMaskedLM.from_pretrained(PRE_TRAINED_DIR)
model_original.eval()

# Load pre-trained model (weights)
model_custom, _ = ErnieForMaskedLM.from_pretrained(PRE_TRAINED_DIR)
model_custom.eval()
pass

In [25]:
#Suppose to predict hensen for idx 8
def eval_sentence(text_a,text_b,model_original,model_custom,tokenizer):
    for masked_idx in range(1,10):
        tokens_tensor,ents_tensor,ent_mask,segments_tensors = process_sentences(text_a,text_b,[masked_idx],name_to_QID,QID_to_eid,eid_to_embeddings,tokenizer,device=device)
    # Predict all tokens
        with torch.no_grad():
            predictions_custom = model_custom(tokens_tensor, ents_tensor, ent_mask, segments_tensors)
            predictions_original =  model_original(tokens_tensor, ents_tensor, ent_mask, segments_tensors)

            # confirm we were able to predict 'henson'
            predicted_index_custom = torch.argmax(predictions_custom[0, masked_idx]).item()
            predicted_token_custom = tokenizer.convert_ids_to_tokens([predicted_index_custom])[0]

            print(f'The predicted token is: {predicted_token_custom}')
            print(f'The predictions between custom and original are equal: {predictions_custom.eq(predictions_original).all()}')

#Load pre-trained model tokenizer (vocabulary)
#Special tokenizer for text and entities
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_DIR)

#Eg: 'Northern Ireland': 'Q26'
name_to_QID = load_name_to_QID(NAME_TO_QID_FILE)
#Eg: {'Q11456633': 4525438, 'Q8863973': 1628631}
QID_to_eid = load_QID_to_eid(QID_TO_EID_FILE)

eid_to_embeddings = torch.load(EID_TO_VEC_FILE)
#Creats a dictionnary of entity index->embeddings
eid_to_embeddings = Embedding.from_pretrained(eid_to_embeddings)

text_a = "Who was Jim Henson ? "
text_b = "Jim Henson was a puppeteer ."
#masked_indices = [8]

#tokens_tensor,ents_tensor,ent_mask,segments_tensors = process_sentences(text_a,text_b,masked_indices,name_to_QID,QID_to_eid,tokenizer)

eval_sentence(text_a,text_b,model_original,model_custom,tokenizer)

The predicted token is who
The predictions between custom and original are equal: True
The predicted token is is
The predictions between custom and original are equal: True
The predicted token is jim
The predictions between custom and original are equal: True
The predicted token is henson
The predictions between custom and original are equal: True
The predicted token is ?
The predictions between custom and original are equal: True
The predicted token is "
The predictions between custom and original are equal: True
The predicted token is jim
The predictions between custom and original are equal: True
The predicted token is henson
The predictions between custom and original are equal: True
The predicted token is was
The predictions between custom and original are equal: True
